## Catboost tuneando un poco los hiperparametros

In [1]:
from sklearn.metrics import roc_auc_score

In [2]:
import numpy as np
import scipy.stats as stats

In [3]:
import xgboost as xgb
import catboost as cb

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
import category_encoders

In [5]:
from sklearn.model_selection import RandomizedSearchCV

In [6]:
kaggle = pd.read_csv('trocafone_kaggle_test.csv')

In [7]:
labels = pd.read_csv('labels_training_set.csv')

In [10]:
features_06 = pd.read_csv('features_test.csv')

In [8]:
features_04 = pd.read_csv('features_04.csv')

In [9]:
repeated = ['busco_productos', 'clickeo_ad', 'hizo_checkout', 'entro_desde_motor',
       'hizo_conversion', 'filtro_busquedas', 'busco_marca', 'visito_sitio',
       'hizo_lead','primer_visita','ultima_visita']

In [11]:
processed = features_06.set_index('person').join(features_04.set_index('person').drop(repeated,axis=1),lsuffix='1')

In [12]:
cat_features = []
for i,col in enumerate(processed.columns):
    if processed[col].dtype.name=='object':
        cat_features.append(i)

In [14]:
predict_features = kaggle.set_index('person').join(processed)

In [13]:
training_features = labels.set_index('person').join(processed)
training_labels = training_features['label']
training_features = training_features.drop(['label'],axis=1)

## Prediccion (score: 0.86547)

In [18]:
xtrain,xtest,ytrain,ytest = train_test_split(training_features.drop(['Unknown','Unknown.1'],axis=1),training_labels)

In [36]:
cb_classifier = cb.CatBoostClassifier(iterations=400,
                                 depth=8,
                                 l2_leaf_reg=100,
                                 random_strength=0.1,
                                 bagging_temperature=0,
                                 border_count=254,
                                learning_rate=0.03)

In [20]:
cat_features2 = []
for i,col in enumerate(xtrain.columns):
    if xtrain[col].dtype.name=='object':
        cat_features2.append(i)

In [34]:
cb_classifier.fit(xtrain,ytrain,cat_features=cat_features2,verbose=0)

In [35]:
cb_classifier.eval_metrics(cb.Pool(xtest,ytest,cat_features=cat_features2),'AUC')

{'AUC': [0.7271656702124907,
  0.7308221617120627,
  0.7406791596049168,
  0.7620844054252088,
  0.7620736956624569,
  0.7619103717804891,
  0.7623267138074723,
  0.770310841939074,
  0.7702724652892127,
  0.7733082367892845,
  0.7732886022242393,
  0.7730877941726395,
  0.7847132416399145,
  0.8200273455942267,
  0.8200929428910826,
  0.8254393457048943,
  0.8346867796011148,
  0.8343297875093822,
  0.8345707571713017,
  0.8439654503053621,
  0.8436316626995921,
  0.8432139819522648,
  0.8427601557556497,
  0.8426401171648046,
  0.8414067094878681,
  0.8418190353538193,
  0.8470596792604552,
  0.8473854345441613,
  0.8499682723278472,
  0.8503681034705879,
  0.8501655104585295,
  0.852340484777411,
  0.8529728070198925,
  0.8525185345831626,
  0.8529661134181725,
  0.8530259095935377,
  0.8527782463298982,
  0.8552736210511097,
  0.8551482275788886,
  0.854382479542122,
  0.8546372826475961,
  0.8543400867312287,
  0.8544520930000098,
  0.8539130349414934,
  0.8544199637117539,
  0.85

In [22]:
roc_auc_score(ytest,cb_classifier.predict_proba(xtest)[:,1])

0.8662912894822097

In [39]:
cb_classifier.fit(training_features.drop(['Unknown','Unknown.1'],axis=1),training_labels,cat_features=cat_features2,verbose=0)

In [40]:
proba_loco=cb_classifier.predict_proba(predict_features.drop(['Unknown','Unknown.1'],axis=1))[:,1]

In [41]:
submit = kaggle.set_index('person')
submit['label'] = proba_loco

In [42]:
submit.to_csv('submit.csv')